In [2]:
import pandas as pd
import numpy as np 

countryName = []
eprtrSectorName = []
EPRTRAnnexIMainActivityLabel = []
FacilityInspireID = []
facilityName = []
City =[]
targetRelease = []
pollutant = []
reportingYear = []
MONTH= []
DAY = []
CONTINENT= []
max_wind_speed =[]
avg_wind_speed= []
min_wind_speed = []
max_temp = []
avg_temp= []
min_temp=[]
FOGDAYS=[]
CITYID = []


url1 = 'http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/first'
url2 = 'http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/second'
url3 = 'http://schneiderapihack-env.eba-3ais9akk.us-east-2.elasticbeanstalk.com/third'
urls = [url1,url2,url3]
for url in urls: 

    from urllib.request import urlopen

    # import json
    import json
    # store the URL in url as 
    # parameter for urlopen
    url = url

    # store the response of URL
    response = urlopen(url)

    # storing the JSON response 
    # from url in data
    data_json = json.loads(response.read())

    
    for jsons in data_json:
        countryName.append(jsons.get('countryName'))
        eprtrSectorName.append(jsons.get('eprtrSectorName'))
        EPRTRAnnexIMainActivityLabel.append(jsons.get('EPRTRAnnexIMainActivityLabel'))
        FacilityInspireID.append(jsons.get('FacilityInspireID'))
        facilityName.append(jsons.get('facilityName'))
        City.append(jsons.get('City'))
        targetRelease.append(jsons.get('targetRelease'))
        pollutant.append(jsons.get('pollutant'))
        reportingYear.append(jsons.get('reportingYear'))
        MONTH.append(jsons.get('MONTH'))
        DAY.append(jsons.get('DAY'))
        CONTINENT.append(jsons.get('CONTINENT'))
        max_wind_speed.append(jsons.get('max_wind_speed'))
        avg_wind_speed.append(jsons.get('avg_wind_speed'))
        min_wind_speed.append(jsons.get('min_wind_speed'))
        max_temp.append(jsons.get('max_temp'))
        avg_temp.append(jsons.get('avg_temp'))
        min_temp.append(jsons.get('min_temp'))
        FOGDAYS.append(jsons.get('DAY WITH FOGS'))
        CITYID.append(jsons.get('CITY ID'))

    
df_json = pd.DataFrame({'countryName':countryName,
                        'eprtrSectorName':eprtrSectorName,
                        'EPRTRAnnexIMainActivityLabel':EPRTRAnnexIMainActivityLabel,
                        'FacilityInspireID':FacilityInspireID,
                        'facilityName':facilityName,
                        'City':City,
                        'targetRelease':targetRelease,
                        'pollutant':pollutant,
                        'reportingYear':reportingYear,
                        'MONTH':MONTH,
                        'DAY':DAY,
                        'CONTINENT':CONTINENT,
                        'max_wind_speed':max_wind_speed,
                        'avg_wind_speed':avg_wind_speed,
                        'min_wind_speed':min_wind_speed,
                        'max_temp':max_temp,
                        'avg_temp':avg_temp,
                        'min_temp': min_temp,
                        'FOGDAYS':FOGDAYS,
                        'CITYID':CITYID})

In [3]:
df_json.rename(columns = {'FOGDAYS':'DAY WITH FOGS'}, inplace = True)

In [4]:
df1 = pd.read_csv(r"C:\Users\Usuario\Desktop\SchneiderHackaton\train1.csv")
df2 = pd.read_csv(r"C:\Users\Usuario\Desktop\SchneiderHackaton\train2.csv",sep=';')

df_csv = pd.concat([df1,df2])
df_total = pd.concat([df_csv,df_json])
df_total.reset_index().drop('index',inplace =True,axis = 1)



In [5]:
df_total['pollutant'].unique()

array(['Carbon dioxide (CO2)', 'Nitrogen oxides (NOX)', 'Methane (CH4)'],
      dtype=object)

In [6]:
dict_map ={'Carbon dioxide (CO2)':1,'Nitrogen oxides (NOX)':0,'Methane (CH4)':2}

df_total['pollutant'] = df_total['pollutant'].map(dict_map)

for columns in df_total.columns:
    try:
        df_total[columns]= df_total[columns].astype(float)
    except:
        'None'

In [7]:
#mas o menos de 5 dias de niebla
def foggy(x):
    if x >5:
        return 1
    else:
        return 0 
    
df_total['FOGGY']= df_total['DAY WITH FOGS'].apply(foggy)
#Es o no es el reino unido, donde parece que tiene una distribucion de pollutant muy distinta a los otros paises
def isuk(x):
    if x == 'United Kingdom':
        return 1
    else:
        return 0 
df_total['ISUK'] = df_total['countryName'].apply(isuk)

In [21]:
df_proc = df_total.drop(columns = ['countryName','FacilityInspireID','facilityName','City','targetRelease','CONTINENT','eprtrSectorName'])
df_proc = df_proc.drop(columns = ['CITY ID','REPORTER NAME','CITYID'])

df_dummed = pd.get_dummies(df_proc)
df_dummed.drop(columns = ['DAY WITH FOGS','reportingYear','MONTH','DAY'],inplace = True)
df_dummed

,pollutant,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,FOGGY,ISUK,"EPRTRAnnexIMainActivityLabel_Chemical installations for the production on an industrial scale of basic inorganic chemicals. Note to reporters, use Level 3 activity e.g. 4(b)(i), in preference to 4(b). Level 2 activity class (i.e. 4(b)) only to be used where Level 3 is not available.",...,EPRTRAnnexIMainActivityLabel_Opencast mining and quarrying,"EPRTRAnnexIMainActivityLabel_Plants for the pre-treatment (operations such as washing, bleaching, mercerisation) or dyeing of fibres or textiles",EPRTRAnnexIMainActivityLabel_Slaughterhouses,EPRTRAnnexIMainActivityLabel_Thermal power stations and other combustion installations,EPRTRAnnexIMainActivityLabel_Treatment and processing intended for the production of food and beverage products from animal raw materials (other than milk),EPRTRAnnexIMainActivityLabel_Treatment and processing intended for the production of food and beverage products from vegetable raw materials,"EPRTRAnnexIMainActivityLabel_Treatment and processing intended for the production of food and beverage products. Note to reporters, use Level 3 activity e.g. 8(b)(i), in preference to 8(b). Level 2 activity class (i.e. 8(b)) only to be used where Level 3 is not available.",EPRTRAnnexIMainActivityLabel_Treatment and processing of milk,EPRTRAnnexIMainActivityLabel_Underground mining and related operations,EPRTRAnnexIMainActivityLabel_Urban waste-water treatment plants
0,1.0,15.118767,14.312541,21.419106,2.864895,4.924169,9.688206,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,19.661550,19.368166,21.756389,5.462839,7.864403,12.023521,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2.0,12.729453,14.701985,17.103930,1.511201,4.233438,8.632193,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,11.856417,16.122584,17.537184,10.970301,10.298348,15.179215,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,2.0,17.111930,20.201604,21.536012,11.772039,11.344078,16.039004,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28496,1.0,13.475988,18.556476,22.852530,13.345801,12.410783,17.148327,0,0,0,...,0,0,0,1,0,0,0,0,0,0
28497,0.0,8.815939,14.461703,20.553781,3.820281,3.763833,5.657107,0,0,0,...,0,0,0,1,0,0,0,0,0,0
28498,2.0,14.793298,16.688049,20.411498,17.285365,18.349798,21.538441,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28499,0.0,14.911317,16.144091,22.647192,6.387199,6.176238,9.269076,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [33]:
from sklearn.model_selection import train_test_split 


X = df_dummed[df_dummed.columns[1:]]
y = df_dummed['pollutant']
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.15,
                                                    random_state=2)
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=250)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6827831386490605


In [23]:
v= df_dummed["avg_wind_speed"]
t= df_dummed["avg_temp"]
df_dummed["Temp_Sen"] = 13.12 + 0.6215 * t - 11.73 * v ** 0.16 + 0.3965 * t * v ** 0.16

In [24]:
df_dummed

,pollutant,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,FOGGY,ISUK,"EPRTRAnnexIMainActivityLabel_Chemical installations for the production on an industrial scale of basic inorganic chemicals. Note to reporters, use Level 3 activity e.g. 4(b)(i), in preference to 4(b). Level 2 activity class (i.e. 4(b)) only to be used where Level 3 is not available.",...,"EPRTRAnnexIMainActivityLabel_Plants for the pre-treatment (operations such as washing, bleaching, mercerisation) or dyeing of fibres or textiles",EPRTRAnnexIMainActivityLabel_Slaughterhouses,EPRTRAnnexIMainActivityLabel_Thermal power stations and other combustion installations,EPRTRAnnexIMainActivityLabel_Treatment and processing intended for the production of food and beverage products from animal raw materials (other than milk),EPRTRAnnexIMainActivityLabel_Treatment and processing intended for the production of food and beverage products from vegetable raw materials,"EPRTRAnnexIMainActivityLabel_Treatment and processing intended for the production of food and beverage products. Note to reporters, use Level 3 activity e.g. 8(b)(i), in preference to 8(b). Level 2 activity class (i.e. 8(b)) only to be used where Level 3 is not available.",EPRTRAnnexIMainActivityLabel_Treatment and processing of milk,EPRTRAnnexIMainActivityLabel_Underground mining and related operations,EPRTRAnnexIMainActivityLabel_Urban waste-water treatment plants,Temp_Sen
0,1.0,15.118767,14.312541,21.419106,2.864895,4.924169,9.688206,0,0,0,...,0,0,0,0,0,0,0,0,0,1.213002
1,0.0,19.661550,19.368166,21.756389,5.462839,7.864403,12.023521,0,0,0,...,0,0,0,0,0,0,0,0,0,4.171225
2,2.0,12.729453,14.701985,17.103930,1.511201,4.233438,8.632193,0,0,0,...,0,0,0,0,0,0,0,0,0,0.298234
3,0.0,11.856417,16.122584,17.537184,10.970301,10.298348,15.179215,0,0,0,...,0,0,1,0,0,0,0,0,0,7.589780
4,2.0,17.111930,20.201604,21.536012,11.772039,11.344078,16.039004,0,0,0,...,0,0,0,0,0,0,0,0,1,8.471996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28496,1.0,13.475988,18.556476,22.852530,13.345801,12.410783,17.148327,0,0,0,...,0,0,1,0,0,0,0,0,0,9.967780
28497,0.0,8.815939,14.461703,20.553781,3.820281,3.763833,5.657107,0,0,0,...,0,0,1,0,0,0,0,0,0,-0.238439
28498,2.0,14.793298,16.688049,20.411498,17.285365,18.349798,21.538441,0,0,0,...,0,0,0,0,0,0,0,0,0,17.536207
28499,0.0,14.911317,16.144091,22.647192,6.387199,6.176238,9.269076,0,0,0,...,0,0,0,0,0,0,0,1,0,2.474728


In [28]:
#Desviacion temp maxima
df_dummed['var_max_temp'] = df_dummed.max_temp - df_dummed.avg_temp
df_dummed['var_max_temp'] = df_dummed['var_max_temp'].apply(lambda x: ((x**2)/2)**0.5)

# Desviacion tipica Temp min
df_dummed['var_min_temp'] = df_dummed.min_temp - df_dummed.avg_temp
df_dummed['var_min_temp'] = df_dummed['var_min_temp'].apply(lambda x: ((x**2)/2)**0.5)

# Desviacion tipica viento max
df_dummed['var_max_wind'] = df_dummed.max_wind_speed - df_dummed.avg_wind_speed
df_dummed['var_max_wind'] = df_dummed['var_max_wind'].apply(lambda x: ((x**2)/2)**0.5)

# Desviacion tipica viento min
df_dummed['var_min_wind'] = df_dummed.min_wind_speed - df_dummed.avg_wind_speed
df_dummed['var_min_wind'] = df_dummed['var_min_wind'].apply(lambda x: ((x**2)/2)**0.5)

In [29]:
df_dummed

,pollutant,max_wind_speed,avg_wind_speed,min_wind_speed,max_temp,avg_temp,min_temp,FOGGY,ISUK,"EPRTRAnnexIMainActivityLabel_Chemical installations for the production on an industrial scale of basic inorganic chemicals. Note to reporters, use Level 3 activity e.g. 4(b)(i), in preference to 4(b). Level 2 activity class (i.e. 4(b)) only to be used where Level 3 is not available.",...,EPRTRAnnexIMainActivityLabel_Treatment and processing intended for the production of food and beverage products from vegetable raw materials,"EPRTRAnnexIMainActivityLabel_Treatment and processing intended for the production of food and beverage products. Note to reporters, use Level 3 activity e.g. 8(b)(i), in preference to 8(b). Level 2 activity class (i.e. 8(b)) only to be used where Level 3 is not available.",EPRTRAnnexIMainActivityLabel_Treatment and processing of milk,EPRTRAnnexIMainActivityLabel_Underground mining and related operations,EPRTRAnnexIMainActivityLabel_Urban waste-water treatment plants,Temp_Sen,var_max_temp,var_min_temp,var_max_wind,var_min_wind
0,1.0,15.118767,14.312541,21.419106,2.864895,4.924169,9.688206,0,0,0,...,0,0,0,0,0,1.213002,1.456126,3.368683,0.570088,5.025100
1,0.0,19.661550,19.368166,21.756389,5.462839,7.864403,12.023521,0,0,0,...,0,0,0,0,0,4.171225,1.698162,2.940940,0.207454,1.688729
2,2.0,12.729453,14.701985,17.103930,1.511201,4.233438,8.632193,0,0,0,...,0,0,0,0,0,0.298234,1.924912,3.110390,1.394791,1.698432
3,0.0,11.856417,16.122584,17.537184,10.970301,10.298348,15.179215,0,0,0,...,0,0,0,0,0,7.589780,0.475143,3.451294,3.016635,1.000273
4,2.0,17.111930,20.201604,21.536012,11.772039,11.344078,16.039004,0,0,0,...,0,0,0,0,1,8.471996,0.302614,3.319814,2.184729,0.943569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28496,1.0,13.475988,18.556476,22.852530,13.345801,12.410783,17.148327,0,0,0,...,0,0,0,0,0,9.967780,0.661157,3.349949,3.592448,3.037769
28497,0.0,8.815939,14.461703,20.553781,3.820281,3.763833,5.657107,0,0,0,...,0,0,0,0,0,-0.238439,0.039915,1.338746,3.992158,4.307750
28498,2.0,14.793298,16.688049,20.411498,17.285365,18.349798,21.538441,0,0,0,...,0,0,0,0,0,17.536207,0.752668,2.254711,1.339791,2.632876
28499,0.0,14.911317,16.144091,22.647192,6.387199,6.176238,9.269076,0,0,0,...,0,0,0,1,0,2.474728,0.149172,2.186966,0.871703,4.598387


In [31]:
df_dummed.drop(columns = ['max_wind_speed','avg_wind_speed','min_wind_speed','max_temp','avg_temp','min_temp'],inplace = True)

In [32]:
df_dummed

,pollutant,FOGGY,ISUK,"EPRTRAnnexIMainActivityLabel_Chemical installations for the production on an industrial scale of basic inorganic chemicals. Note to reporters, use Level 3 activity e.g. 4(b)(i), in preference to 4(b). Level 2 activity class (i.e. 4(b)) only to be used where Level 3 is not available.","EPRTRAnnexIMainActivityLabel_Chemical installations for the production on an industrial scale of basic inorganic chemicals: Acids, such as chromic acid, hydrofluoric acid, phosphoric acid, nitric acid, hydrochloric acid, sulphuric acid, oleum, sulphurous acids","EPRTRAnnexIMainActivityLabel_Chemical installations for the production on an industrial scale of basic inorganic chemicals: Bases, such as ammonium hydroxide, potassium hydroxide, sodium hydroxide","EPRTRAnnexIMainActivityLabel_Chemical installations for the production on an industrial scale of basic inorganic chemicals: Gases, such as ammonia, chlorine or hydrogen chloride, fluorine or hydrogen fluoride, carbon oxides, sulphur compounds, nitrogen oxides, hydrogen, sulphur dioxide, carbonyl chloride","EPRTRAnnexIMainActivityLabel_Chemical installations for the production on an industrial scale of basic inorganic chemicals: Non-metals, metal oxides or other inorganic compounds such as calcium carbide, silicon, silicon carbide","EPRTRAnnexIMainActivityLabel_Chemical installations for the production on an industrial scale of basic inorganic chemicals: Salts, such as ammonium chloride, potassium chlorate, potassium carbonate, sodium carbonate, perborate, silver nitrate","EPRTRAnnexIMainActivityLabel_Chemical installations for the production on an industrial scale of basic organic chemicals. Note to reporters, use Level 3 activity e.g. 4(a)(iii), in preference to 4(a). Level 2 activity class (i.e. 4(a)) only to be used where Level 3 is not available.",...,EPRTRAnnexIMainActivityLabel_Treatment and processing intended for the production of food and beverage products from vegetable raw materials,"EPRTRAnnexIMainActivityLabel_Treatment and processing intended for the production of food and beverage products. Note to reporters, use Level 3 activity e.g. 8(b)(i), in preference to 8(b). Level 2 activity class (i.e. 8(b)) only to be used where Level 3 is not available.",EPRTRAnnexIMainActivityLabel_Treatment and processing of milk,EPRTRAnnexIMainActivityLabel_Underground mining and related operations,EPRTRAnnexIMainActivityLabel_Urban waste-water treatment plants,Temp_Sen,var_max_temp,var_min_temp,var_max_wind,var_min_wind
0,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1.213002,1.456126,3.368683,0.570088,5.025100
1,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4.171225,1.698162,2.940940,0.207454,1.688729
2,2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.298234,1.924912,3.110390,1.394791,1.698432
3,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,7.589780,0.475143,3.451294,3.016635,1.000273
4,2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,8.471996,0.302614,3.319814,2.184729,0.943569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28496,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,9.967780,0.661157,3.349949,3.592448,3.037769
28497,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,-0.238439,0.039915,1.338746,3.992158,4.307750
28498,2.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,17.536207,0.752668,2.254711,1.339791,2.632876
28499,0.0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,2.474728,0.149172,2.186966,0.871703,4.598387


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800]}

''' # Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)'''

# Grid search utilizado para saber cuales eran los mejores parametros.... tarda muchisimo en ejecutar y no gana tanta precision,
#{'n_estimators': 1600, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 90, 'bootstrap': True}